In [18]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [19]:
data_GBP_TR = pd.read_csv("GBP_TRY.csv")
data_GBP_TR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1297 entries, 0 to 1296
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ﻿"Tarih"  1297 non-null   object
 1   Şimdi     1297 non-null   object
 2   Açılış    1294 non-null   object
 3   Yüksek    1294 non-null   object
 4   Düşük     1294 non-null   object
 5   Hac.      1033 non-null   object
 6   Fark %    1294 non-null   object
dtypes: object(7)
memory usage: 71.1+ KB


In [20]:
df = data_GBP_TR.drop(['Açılış','Yüksek','Düşük','Hac.','Fark %'], axis=1)

In [21]:
df.head(10)

,"﻿""Tarih""",Şimdi
0,2022-12-15,22.701371633949773
1,2022-12-15,22.70350298361561
2,2022-12-15,22.704561108354255
3,15.12.2022,"22,6894"
4,14.12.2022,"23,1374"
5,13.12.2022,"22,9513"
6,12.12.2022,"22,8621"
7,09.12.2022,"22,8559"
8,08.12.2022,"22,7930"
9,07.12.2022,"22,7334"


In [22]:
df['Şimdi'] = df['Şimdi'].apply(lambda x: float(x.split()[0].replace(',', '.')))
df
# There are 4 same dates which are 15/12/2022 because i tried to do that i wanted to add new dates as tomorrow.

,"﻿""Tarih""",Şimdi
0,2022-12-15,22.701372
1,2022-12-15,22.703503
2,2022-12-15,22.704561
3,15.12.2022,22.689400
4,14.12.2022,23.137400
...,...,...
1292,05.01.2018,5.070900
1293,04.01.2018,5.079600
1294,03.01.2018,5.110500
1295,02.01.2018,5.118200


In [23]:
# univariate data preparation
from numpy import array
 
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			 break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
# define input sequence
raw_seq = df['Şimdi']
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[22.70137163 22.70350298 22.70456111 22.6894     23.1374    ] 22.9513
[22.70350298 22.70456111 22.6894     23.1374     22.9513    ] 22.8621
[22.70456111 22.6894     23.1374     22.9513     22.8621    ] 22.8559
[22.6894 23.1374 22.9513 22.8621 22.8559] 22.793
[23.1374 22.9513 22.8621 22.8559 22.793 ] 22.7334
[22.9513 22.8621 22.8559 22.793  22.7334] 22.6039
[22.8621 22.8559 22.793  22.7334 22.6039] 22.6973
[22.8559 22.793  22.7334 22.6039 22.6973] 22.891
[22.793  22.7334 22.6039 22.6973 22.891 ] 22.8066
[22.7334 22.6039 22.6973 22.891  22.8066] 22.4115
[22.6039 22.6973 22.891  22.8066 22.4115] 22.2773
[22.6973 22.891  22.8066 22.4115 22.2773] 22.2543
[22.891  22.8066 22.4115 22.2773 22.2543] 22.475
[22.8066 22.4115 22.2773 22.2543 22.475 ] 22.5229
[22.4115 22.2773 22.2543 22.475  22.5229] 22.4242
[22.2773 22.2543 22.475  22.5229 22.4242] 22.112
[22.2543 22.475  22.5229 22.4242 22.112 ] 22.001
[22.475  22.5229 22.4242 22.112  22.001 ] 22.091
[22.5229 22.4242 22.112  22.001  22.091 ] 22.0

In [24]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)
# demonstrate prediction
x_input = array([[df['Şimdi'][0],df['Şimdi'][1],df['Şimdi'][2],df['Şimdi'][3],df['Şimdi'][4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

Epoch 1/200
41/41 [==============================] - 3s 36ms/step - loss: 26.4859
Epoch 2/200
41/41 [==============================] - 1s 35ms/step - loss: 1.0922
Epoch 3/200
41/41 [==============================] - 1s 35ms/step - loss: 0.2145
Epoch 4/200
41/41 [==============================] - 1s 36ms/step - loss: 0.1843
Epoch 5/200
41/41 [==============================] - 1s 35ms/step - loss: 0.1698
Epoch 6/200
41/41 [==============================] - 1s 36ms/step - loss: 0.1686
Epoch 7/200
41/41 [==============================] - 1s 35ms/step - loss: 0.1539
Epoch 8/200
41/41 [==============================] - 2s 36ms/step - loss: 0.1536
Epoch 9/200
41/41 [==============================] - 2s 37ms/step - loss: 0.1463
Epoch 10/200
41/41 [==============================] - 1s 36ms/step - loss: 0.1421
Epoch 11/200
41/41 [==============================] - 1s 35ms/step - loss: 0.1480
Epoch 12/200
41/41 [==============================] - 1s 36ms/step - loss: 0.1510
Epoch 13/200
41/41 [====

In [34]:
print("Prediction:",round(yhat[0][0],3), "Sterlin/TRY")

Prediction: 23.277 Sterlin/TRY
